# Jiboa Case Study

This code is prepared to Run the distributed model for jiboa rover in El Salvador
wher the catchment is consisted os a ustream lake and a volcanic area
-   you have to make the root directory to the examples folder to enable the code
    from reading input files

In [43]:
! pip install gdal
! pip install affine
! pip install fiona
! pip install proj
! pip install pyproj
! pip install pandas
! pip install geopandas
! pip install matplotlib-base
! pip install python
! pip install scipy
! pip install shapely
! pip install statsmodels
! pip install rasterio
! pip install rasterstats
! pip install oasis
! pip install netCDF4
! pip install scikit-learn
! pip install scikit-image
! pip install ecmwf-api-client
! pip install joblib
!pip install git+https://github.com/MAfarrag/HAPI.git

ERROR: Could not find a version that satisfies the requirement matplotlib-base (from versions: none)
ERROR: No matching distribution found for matplotlib-base
ERROR: Could not find a version that satisfies the requirement python (from versions: none)
ERROR: No matching distribution found for python
  Cloning https://github.com/MAfarrag/HAPI.git to /tmp/pip-req-build-tjyqbr5z
  Running command git clone -q https://github.com/MAfarrag/HAPI.git /tmp/pip-req-build-tjyqbr5z
  Created wheel for HAPI-Nile: filename=HAPI_Nile-1.0.4-cp37-none-any.whl size=16849817 sha256=f186987e4dd46c06739f3319f1577b651a487b5bdf2e1e5e9f8882522a487359
  Stored in directory: /tmp/pip-ephem-wheel-cache-bav1y46k/wheels/01/64/ed/1b6002951ba52502516f59c8ec8081033e6c0ec59dff2c4424
Successfully built HAPI-Nile


In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Download Data
For the data of this case study you have to download this folder [Jiboa Data](https://drive.google.com/drive/folders/1uyd9mH8pHHUKK9l3bc7QXlsu4EPwy3Mr?usp=sharing) from Google Drive and set it as the working directory instead of the Path defined in the next cell

## Import modules

In [46]:
#libraries
import numpy as np
import datetime as dt

# HAPI modules
from Hapi.run import Run
from Hapi.catchment import Catchment, Lake
import Hapi.rrm.hbv as HBV
import Hapi.rrm.hbv_lake as HBVLake
import Hapi.sm.performancecriteria as Pf

In [48]:
import os
Path = "/content/drive/My Drive/Hapi/Jiboa/El Salvador/"

os.chdir(Path)

In [49]:
res = 4000

In [54]:
# the beginning of the simulation of the calibration data
PrecPath = Path + "inputs/Hapi/meteodata/"+str(res)+"/calib/"
Evap_Path = Path + "inputs/Hapi/meteodata/"+str(res)+"/calib/"
TempPath = Path + "inputs/Hapi/meteodata/"+str(res)+"/calib/"
FlowAccPath = Path + "inputs/Hapi/GIS/"+str(res)+"_matched/acc"+str(res)+".tif"
FlowDPath = Path + "inputs/Hapi/GIS/"+str(res)+"_matched/fd"+str(res)+".tif"
ParPath = Path + "inputs/Hapi/meteodata/"+str(res)+"/parameters/"
# Lake
LakeMeteoPath = Path + "inputs/Hapi/meteodata/lakedata.csv"
LakeParametersPath = Path + "inputs/Hapi/meteodata/"+str(res)+"/Lakeparameters.txt"
GaugesPath = Path + "inputs/Hapi/meteodata/Gauges/"
SaveTo = Path + "results/"

In [55]:
AreaCoeff = 227.31
InitialCond = np.loadtxt("inputs/Hapi/meteodata/Initia-jiboa.txt", usecols=0).tolist()
Snow = 0

Sdate = '2012-06-14 19:00:00'
# Edate = '2014-11-17 00:00:00'
Edate = '2013-12-23 00:00:00'
name = "Jiboa"
Jiboa = Catchment(name, Sdate, Edate, SpatialResolution = "Distributed",
              TemporalResolution = "Hourly", fmt='%Y-%m-%d %H:%M:%S')
Jiboa.readRainfall(PrecPath)
Jiboa.readTemperature(TempPath)
Jiboa.readET(Evap_Path)
Jiboa.readFlowAcc(FlowAccPath)
Jiboa.readFlowDir(FlowDPath)
Jiboa.readParameters(ParPath, Snow)

Jiboa.readLumpedModel(HBV, AreaCoeff, InitialCond)

AssertionError: ignored

In [ ]:

#p2=[24, 1530]
#init_st=[0,5,5,5,0]
init_st = np.loadtxt("inputs/Hapi/meteodata/Initia-jiboa.txt", usecols=0).tolist()
snow = 0


# lake meteorological data
ind = pd.date_range(start, end, freq = "H" )
lakedata = pd.read_csv("inputs/Hapi/meteodata/lakedata.csv", index_col = 0)
lakedata.index = ind
lakeCalib = lakedata.loc[start:calib_end]
lakeValid = lakedata.loc[calib_end:end]
# convert the dataframe into array
lakeCalibArray = lakeCalib.values
# take only the plake, et, t and tm columns and exclude the last column
lakeCalibArray = lakeCalibArray[:,0:-1]

# where the lake discharges its flow (give the indices of the cell)
lakecell = [2,1]    # 4km
#lakecell = [4,2]    # 2km
#lakecell = [10,4]    # 1km
#lakecell = [19,10]    # 500m

LakeParameters = np.loadtxt("inputs/Hapi/meteodata/4000/Lakeparameters.txt").tolist()
StageDischargeCurve = np.loadtxt("inputs/Hapi/meteodata/curve.txt")
p2 = [1, 227.31, 133.98, 70.64]
Lake_init_st = np.loadtxt("inputs/Hapi/meteodata/Initia-lake.txt", usecols=0).tolist()

# Run the model

In [ ]:
os.getcwd()

'/content/drive/My Drive'

In [ ]:
os.listdir()

['Hapi', 'Jiboa.mxd', 'web application', 'processing', 'inputs', 'results.zip']

In [ ]:
Sim =pd.DataFrame(index = lakeCalib.index)
st, Sim['Q'], q_uz_routed, q_lz_trans = runHAPIwithLake(HBV, Paths, ParPath, p2, init_st,
                                                     snow, lakeCalibArray, StageDischargeCurve,
                                                     LakeParameters, lakecell,Lake_init_st)

### Evaluate model performance

In [ ]:
WS = {}
WS['type'] = 1
WS['N'] = 3
ModelMetrics=dict()
ModelMetrics['CalibErrorHf']=Pf.RMSEHF(lakeCalib['Q'],Sim['Q'],WS['type'],WS['N'],0.75)
ModelMetrics['CalibErrorLf']=Pf.RMSELF(lakeCalib['Q'],Sim['Q'],WS['type'],WS['N'],0.75)
ModelMetrics['CalibNSEHf']=Pf.NSE(lakeCalib['Q'],Sim['Q'])
ModelMetrics['CalibNSELf']=Pf.NSE(np.log(lakeCalib['Q']),np.log(Sim['Q']))
ModelMetrics['CalibRMSE']=Pf.RMSE(lakeCalib['Q'],Sim['Q'])
ModelMetrics['CalibKGE']=Pf.KGE(lakeCalib['Q'],Sim['Q'])
ModelMetrics['CalibWB']=Pf.WB(lakeCalib['Q'],Sim['Q'])
print("RMSE(HF) = " + str(round(ModelMetrics['CalibErrorHf'],2)))
print("RMSE(LF) = " + str(round(ModelMetrics['CalibErrorLf'],2)))
print("RMSE = " + str(round(ModelMetrics['CalibRMSE'],2)))
print("NSE(HF) = " + str(round(ModelMetrics['CalibNSEHf'],2)))
print("NSE(LF) = " + str(round(ModelMetrics['CalibNSELf'],2)))
print("KGE = " + str(round(ModelMetrics['CalibKGE'],2)))
print("WB = " + str(round(ModelMetrics['CalibWB'],2)))

In [ ]:
plt.figure(50,figsize=(15,8))
Sim.Q.plot(color=[(0,0.3,0.7)],linewidth=2.5,label="Simulated data", zorder = 10)
ax1=lakeCalib['Q'].plot(color='#DC143C',linewidth=2.8,label='Observed data')
ax1.annotate("Model performance" ,xy=('2012-12-01 00:00:00',20),fontsize=15)
ax1.annotate("RMSE = " + str(round(ModelMetrics['CalibRMSE'],3)),xy=('2012-12-01 00:00:00',20-1.5),fontsize=15)
ax1.annotate("NSE = " + str(round(ModelMetrics['CalibNSEHf'],2)),xy=('2012-12-01 00:00:00',20-3),fontsize=15)
plt.legend()

# Store the result into rasters

In [ ]:
# create list of names
src=gdal.Open(FlowAccPath)

index=pd.date_range(start,calib_end,freq="1H")

resultspath="results/upper_zone_discharge/4000/"
names=[resultspath+str(i)[:-6] for i in index]
names=[i.replace("-","_") for i in names]
names=[i.replace(" ","_") for i in names]
names=[i+".tif" for i in names]


Raster.RastersLike(src,q_uz_routed[:,:,:-1],names)
